In [10]:
# ==============================================================================
# CELLA 1: SETUP, LIBRERIE E CONTROL CENTER
# ==============================================================================

# Installazione librerie base
!pip install -q transformers torch datasets

import os
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

# --- CONTROL CENTER ---
CONFIG = {
    # Percorsi di Input (Creati nel Notebook 1)
    "SPLITS_DIR": os.path.join("DATASET_ITA", "PROCESSED_DATA", "splits"),
    "HF_OUT_DIR": os.path.join("DATASET_ITA", "PROCESSED_DATA", "HF_DATASETS"),
    "FRAMES_BASE_DIR": os.path.join("DATASET_ITA", "PROCESSED_DATA", "frames"),
    
    # Label Mapping per M2 (Classification)
    "LABEL_MAP": {
        "flaming": 1,
        "denigration": 2,
        "sexual": 3,
        "racism": 4
    },
    
    # SEED (Per riproducibilità mescolamento)
    "SAMPLING_SEED": 123
}

# Creazione cartelle output
os.makedirs(CONFIG["HF_OUT_DIR"], exist_ok=True)

# Setup Seed Globale
np.random.seed(CONFIG["SAMPLING_SEED"])
import random
random.seed(CONFIG["SAMPLING_SEED"])

print(f"✅ Setup completato.")
print(f"📂 Output HuggingFace: {CONFIG['HF_OUT_DIR']}")

✅ Setup completato.
📂 Output HuggingFace: DATASET_ITA/PROCESSED_DATA/HF_DATASETS


In [11]:
# ==============================================================================
# CELLA 2: CARICAMENTO DATASET MASTER
# ==============================================================================

print("⏳ Caricamento Dataset Master (output del Notebook 1)...")

# Dizionario per iterare sui file
split_files = {
    "TRAIN": "master_train.csv",
    "VAL":   "master_val.csv",
    "TEST":  "master_test.csv"
}

datasets = {}

try:
    for split_name, filename in split_files.items():
        file_path = os.path.join(CONFIG["SPLITS_DIR"], filename)
        
        # Carichiamo specificando il separatore ';' e che video_id è stringa
        df = pd.read_csv(file_path, sep=';', dtype={'video_id': str})
        
        # Normalizzazione Label: tutto minuscolo e senza spazi extra
        if 'Type' in df.columns:
            df['Type'] = df['Type'].astype(str).str.lower().str.strip()
            
        datasets[split_name] = df
        print(f"   ✅ {split_name:<5}: Caricato con successo ({len(df)} righe)")

except FileNotFoundError as e:
    print(f"\n❌ ERRORE: Non trovo il file {e.filename}")
    print("   Suggerimento: Hai eseguito il Notebook 1 e salvato i file in PROCESSED_DATA/splits?")
    raise

# Assegnazione alle variabili globali per comodità
df_train = datasets["TRAIN"]
df_val   = datasets["VAL"]
df_test  = datasets["TEST"]

# Report preliminare sul Train Set (quello su cui lavoreremo ora)
print(f"\n📊 ANALISI PRELIMINARE TRAIN SET:")
print(f"   🔹 Totale Commenti: {len(df_train)}")
print(f"   🔹 Video Unici:     {df_train['video_id'].nunique()}")
print("\n   📉 Distribuzione Classi Originale:")
print(df_train['Type'].value_counts())

⏳ Caricamento Dataset Master (output del Notebook 1)...
   ✅ TRAIN: Caricato con successo (3466 righe)
   ✅ VAL  : Caricato con successo (710 righe)
   ✅ TEST : Caricato con successo (855 righe)

📊 ANALISI PRELIMINARE TRAIN SET:
   🔹 Totale Commenti: 3466
   🔹 Video Unici:     28

   📉 Distribuzione Classi Originale:
Type
none           2597
denigration     616
sexual          185
flaming          50
racism           18
Name: count, dtype: int64


In [12]:
# ==============================================================================
# CELLA UNICA: DATASET M1 CREATION (UNDERSAMPLING INTELLIGENTE 'NONE')
# ==============================================================================

import re

print("🚀 PREPARAZIONE DATASET M1 - UNDERSAMPLING AVANZATO (None only)")

TEXT_COL = "Comment"

# --- 1. Separazione iniziale ---
df_bully = df_train[df_train["Type"] != "none"].copy()
df_none = df_train[df_train["Type"] == "none"].copy()

start_none = len(df_none)
print(f"🔹 Commenti 'None' iniziali: {start_none}")

# --- 2. Definizione filtri ---

spam_keywords = {
    "first", "primo", "second", "secondo",
    "like", "likes", "follow", "segui",
    "sub", "iscriviti", "views", "visualizzazioni",
    "edit", "parte", "pt", "up", "upp", "pls", "plz", "please",
    "push", "boost", "spam", "link", "bio", "profilo", "dm", "priv", "pvt"
}

social_noise_words = {
    "ahah", "haha", "lol", "ok", "boh", "mah",
    "nice", "bella", "top", "grande"
}

def tokenize(text):
    return re.findall(r"\w+", text.lower())

def is_exact_spam(text):
    if not isinstance(text, str):
        return True
    clean = re.sub(r"[^\w\s]", "", text.lower().strip())
    return clean in spam_keywords

def is_only_emoji_or_punct(text):
    if not isinstance(text, str):
        return True
    return not any(char.isalnum() for char in text)

def is_too_short(text, min_tokens=3):
    return len(tokenize(text)) < min_tokens

def is_spam_containment_short(text, max_tokens=3):
    tokens = tokenize(text)
    if len(tokens) > max_tokens:
        return False
    return any(tok in spam_keywords for tok in tokens)

def is_social_noise(text):
    tokens = tokenize(text)
    return len(tokens) == 1 and tokens[0] in social_noise_words

def is_low_quality_none(text):
    return (
        is_exact_spam(text) or
        is_only_emoji_or_punct(text) or
        is_too_short(text) or
        is_spam_containment_short(text) or
        is_social_noise(text)
    )

# --- 3. Applicazione filtri ---
print("🧹 Applicazione filtri di qualità semantica sui commenti 'None'...")
mask_keep = ~df_none[TEXT_COL].apply(is_low_quality_none)
df_none_clean = df_none[mask_keep].copy()

removed = start_none - len(df_none_clean)
perc_removed = (removed / start_none) * 100

print(f"   🔻 Rimossi: {removed} commenti ({perc_removed:.1f}%)")
print(f"   ✅ Commenti 'None' validi rimasti: {len(df_none_clean)}")

# --- 4. Assegnazione label binaria ---
df_none_clean["binary_label"] = 0
df_bully["binary_label"] = 1

# --- 5. Merge e shuffle ---
df_train_M1 = pd.concat([df_none_clean, df_bully]).sample(
    frac=1,
    random_state=CONFIG["SAMPLING_SEED"]
).reset_index(drop=True)

# --- 6. Report finale ---
n_safe = len(df_none_clean)
n_bully = len(df_bully)
total = len(df_train_M1)

print("\n📊 DISTRIBUZIONE FINALE DATASET M1")
print(f"   🧮 Totale righe: {total}")
print(f"   🟢 Safe (0):  {n_safe} ({n_safe/total:.1%})")
print(f"   🔴 Bully (1): {n_bully} ({n_bully/total:.1%})")
print(f"   ⚖️  Rapporto Safe/Bully: {n_safe / n_bully:.2f} : 1")

# --- 7. Check di integrità ---
assert not df_train_M1["video_id"].isnull().any(), "Errore: video_id nulli"
assert TEXT_COL in df_train_M1.columns, "Errore: colonna testo mancante"

print("✅ Dataset M1 pronto per il training")


🚀 PREPARAZIONE DATASET M1 - UNDERSAMPLING AVANZATO (None only)
🔹 Commenti 'None' iniziali: 2597
🧹 Applicazione filtri di qualità semantica sui commenti 'None'...
   🔻 Rimossi: 1093 commenti (42.1%)
   ✅ Commenti 'None' validi rimasti: 1504

📊 DISTRIBUZIONE FINALE DATASET M1
   🧮 Totale righe: 2373
   🟢 Safe (0):  1504 (63.4%)
   🔴 Bully (1): 869 (36.6%)
   ⚖️  Rapporto Safe/Bully: 1.73 : 1
✅ Dataset M1 pronto per il training


In [13]:
# ==============================================================================
# CELLA 4: DATASET M2 (STRATEGIA SCIENTIFICA: SEMANTIC AUG + SOFT BOOST 2x)
# ==============================================================================
import pandas as pd
import numpy as np

print("\n⚙️ COSTRUZIONE DATASET M2 (Classification)...")

# 1. Filtro Base: Teniamo solo i commenti offensivi dal TRAIN
df_train_M2 = df_train[df_train['Type'] != 'none'].copy()

# Inizializziamo subito le colonne di supporto per evitare errori di tipo misto
df_train_M2['is_synthetic'] = False
df_train_M2['aug_method'] = 'original'

print(f"   🔹 Base M2 (Solo Offensivi Reali): {len(df_train_M2)} righe")

# --- FASE A: AUGMENTATION SEMANTICA ---
AUG_MAP = {
    "il sardo è più comprensibile": ["persino il calabrese si capisce più di te", "a confronto il sardo sembra italiano perfetto", "parli peggio di un sardo stretto"],
    "il barese è più comprensibile": ["anche il barese si capisce meglio di te", "il barese è molto più chiaro del tuo modo di parlare"],
    "sto rivalutando il sardo": ["a confronto sto rivalutando il sardo", "dopo averti sentito il sardo sembra chiaro"],
    "pari na zingara": ["sembri proprio una zingara", "pari davvero una zingara", "una zingarella"],
    "DAI VAI A PAGARE LE PIZZE": ["vai a lavorare invece di scroccare come una zingara", "paga le pizze invece di scroccare, zingara"],
    "sposa Putin": ["vai a vivere con Putin", "vedrei bene Putin per te...", "sposati Putin e sparisci"],
    "ma ti rendi conto di essere ridicola": ["ti rendi conto di quanto sei ridicola?", "ma ti accorgi di essere ridicola?"],
    "sempre ridicolo è": ["ridicoloo", "sempre ridicolo"],
    "ma chi ti conosce": ["ma chi ti conosce davvero?", "ma chi vuoi che ti conosca"],
    "Ma solo io la vedo cretina": ["sono l’unico a vederla cretina?", "solo io la trovo cretina?"],
    "che morta di fame": ["proprio una morta di fame", "una vera morta di fame"],
    "vai bene come mocio": ["saresti utile come mocio", "vai bene giusto come mocio"],
    "Influencer di sto cazzo": ["influencer di merda, non siete nessuno!", "siete dei pezzenti, altro che influencer"],
    "foto del suo fondoschiena": ["solo foto del suo fondoschiena?", "ancora foto del suo sedere?"],
    "fa vedere il culo": ["influencer vuol dire mostrare il culo in ogni video", "fa vedere il sedere in ogni video"],
    "il tuo latoB tanto vale": ["il tuo lato B vale giusto una pizza", "per quel lato B non pagherei nulla"],
    "se ti pago la pizza": ["se ti offro la pizza cosa mi dai in cambio?", "se pago la pizza per te cosa ottengo?"]
}

new_rows = []
print("\n🚀 [1/2] Avvio Augmentation Semantica...")

for search_key, variants in AUG_MAP.items():
    # Cerca la frase nei commenti (case insensitive)
    mask = df_train_M2['Comment'].astype(str).str.lower().str.contains(search_key.lower().strip(), regex=False)
    matches = df_train_M2[mask]
    
    if len(matches) > 0:
        # Prende il primo match per copiare i metadati
        original_row = matches.iloc[0]
        for v in variants:
            new_rows.append({
                'Comment': v,
                'Type': original_row['Type'],
                'video_id': original_row['video_id'],
                'is_synthetic': True,
                'aug_method': 'semantic_rule'
            })

if new_rows:
    df_aug_semantic = pd.DataFrame(new_rows)
    df_train_M2 = pd.concat([df_train_M2, df_aug_semantic], ignore_index=True)
    print(f"   ✅ Aggiunte {len(df_aug_semantic)} varianti semantiche.")
else:
    print("   ⚠️ Nessuna variante semantica generata.")

# --- FASE B: SOFT OVERSAMPLING 2x ---
print("\n🚀 [2/2] Avvio Soft Oversampling (Solo Classi Rare)...")

RARE_CLASSES = ['racism', 'flaming']
MULTIPLIER = 2 

oversample_rows = []
# Calcoliamo i conteggi sul dataset corrente (che include già l'augmentation semantica!)
counts = df_train_M2['Type'].value_counts()
print(f"   📊 Pre-Oversampling: Racism={counts.get('racism',0)}, Flaming={counts.get('flaming',0)}")

# Iteriamo solo sulle righe ORIGINALI (non quelle appena create) per evitare di duplicare i duplicati all'infinito
# Filtriamo: is_synthetic == False
originals_only = df_train_M2[df_train_M2['is_synthetic'] == False]

for _, row in originals_only.iterrows():
    if row['Type'] in RARE_CLASSES:
        # Creiamo copie
        for _ in range(MULTIPLIER - 1):
            row_copy = row.copy()
            # Qui forziamo il casting a object/bool se necessario, ma dato che creiamo un nuovo DF alla fine, Pandas gestirà i tipi
            row_copy['is_synthetic'] = True
            row_copy['aug_method'] = 'soft_oversampling'
            oversample_rows.append(row_copy)

if oversample_rows:
    df_boost = pd.DataFrame(oversample_rows)
    df_train_M2_FINAL = pd.concat([df_train_M2, df_boost], ignore_index=True)
    print(f"   ✅ Aggiunte {len(df_boost)} righe di supporto (Moltiplicatore {MULTIPLIER}x).")
else:
    df_train_M2_FINAL = df_train_M2.copy()

# Shuffle finale
df_train_M2_FINAL = df_train_M2_FINAL.sample(frac=1, random_state=CONFIG["SAMPLING_SEED"]).reset_index(drop=True)

print(f"\n✅ DATASET M2 COMPLETATO.")
print(f"   Totale Righe: {len(df_train_M2_FINAL)}")
print(f"   Distribuzione Classi Finale:\n{df_train_M2_FINAL['Type'].value_counts()}")

# Check Multimodale
if 'video_id' not in df_train_M2_FINAL.columns:
    raise ValueError("❌ ERRORE CRITICO: video_id perso!")


⚙️ COSTRUZIONE DATASET M2 (Classification)...
   🔹 Base M2 (Solo Offensivi Reali): 869 righe

🚀 [1/2] Avvio Augmentation Semantica...
   ✅ Aggiunte 30 varianti semantiche.

🚀 [2/2] Avvio Soft Oversampling (Solo Classi Rare)...
   📊 Pre-Oversampling: Racism=26, Flaming=64
   ✅ Aggiunte 68 righe di supporto (Moltiplicatore 2x).

✅ DATASET M2 COMPLETATO.
   Totale Righe: 967
   Distribuzione Classi Finale:
Type
denigration    616
sexual         193
flaming        114
racism          44
Name: count, dtype: int64


In [14]:
# ==============================================================================
# CELLA 5: EXPORT DATASETS (M1 e M2) IN FORMATO HUGGING FACE
# ==============================================================================
from datasets import Dataset, DatasetDict
import os

# Recuperiamo i percorsi dalla CONFIG globale
HF_OUT_DIR = CONFIG["HF_OUT_DIR"]
FRAMES_BASE_DIR = CONFIG["FRAMES_BASE_DIR"]

# Mapping M2 (Classification)
LABEL_MAP_M2 = CONFIG["LABEL_MAP"]

def row_to_conversation(row, mode="detection"):
    """
    Converte una riga in formato Chat.
    Gestisce automaticamente la mancanza di 'binary_label' in Val/Test.
    """
    video_id = str(row['video_id'])
    text_comment = row['Comment']
    
    # 1. Recupero Frame
    video_folder = os.path.join(FRAMES_BASE_DIR, video_id)
    image_paths = []
    
    if os.path.exists(video_folder):
        frames = sorted([f for f in os.listdir(video_folder) if f.lower().endswith(".jpg")])
        frames = frames[:3] 
        image_paths = [os.path.abspath(os.path.join(video_folder, f)) for f in frames]
    
    if not image_paths: return None 

    # 2. USER CONTENT (DATA ONLY)
    user_content = []
    for img_path in image_paths:
        user_content.append({"type": "image", "image": img_path})
    
    # Prompt Pulito (Il system prompt sarà aggiunto nel training)
    clean_text = f"Commento: \"{text_comment}\""
    user_content.append({"type": "text", "text": clean_text})

    # 3. LABEL ASSISTANT (LOGICA CORRETTA)
    if mode == "detection":
        # CASO M1:
        # Se abbiamo già la colonna binaria (Training Set), usiamola.
        if 'binary_label' in row:
            label = str(int(row['binary_label']))
        else:
            # Fallback per Val/Test che non hanno la colonna 'binary_label':
            # Se Type è 'none' -> 0, altrimenti -> 1
            label = "0" if row['Type'] == 'none' else "1"
            
    elif mode == "classification":
        # CASO M2:
        if row['Type'] not in LABEL_MAP_M2: return None
        label = str(LABEL_MAP_M2[row['Type']])
    else:
        return None

    # Struttura Chat Finale
    return [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": [{"type": "text", "text": label}]}
    ]

def create_and_save_hf_dataset(df_train, df_val, df_test, task_name, mode):
    print(f"\n🏗️  Costruzione Dataset HF: {task_name} ({mode})...")
    
    splits = {"train": df_train, "val": df_val, "test": df_test}
    ds_dict = {}
    
    for split_name, df in splits.items():
        if df is None or len(df) == 0: continue
            
        data_list = []
        skipped = 0
        
        for _, row in df.iterrows():
            msgs = row_to_conversation(row, mode=mode)
            if msgs:
                data_list.append({"messages": msgs}) 
            else:
                skipped += 1
        
        if data_list:
            ds_dict[split_name] = Dataset.from_list(data_list)
            print(f"   🔹 {split_name.upper()}: {len(ds_dict[split_name])} esempi (Skipped: {skipped})")
        else:
            print(f"   ⚠️ {split_name.upper()}: Nessun dato valido generato.")

    # Salvataggio
    final_ds = DatasetDict(ds_dict)
    save_path = os.path.join(HF_OUT_DIR, task_name)
    final_ds.save_to_disk(save_path)
    print(f"   ✅ Salvato in: {save_path}")

# --- ESECUZIONE ---
print("📦 AVVIO ESPORTAZIONE HUGGING FACE DATASETS...")

# 1. TASK 1: DETECTION (M1)
create_and_save_hf_dataset(
    df_train_M1, df_val, df_test, 
    task_name="M1_detection", 
    mode="detection"
)

# 2. TASK 2: CLASSIFICATION (M2)
df_val_m2 = df_val[df_val['Type'] != 'none'].copy()
df_test_m2 = df_test[df_test['Type'] != 'none'].copy()

create_and_save_hf_dataset(
    df_train_M2_FINAL, df_val_m2, df_test_m2, 
    task_name="M2_classification", 
    mode="classification"
)

print(f"\n🎉 FINE NOTEBOOK 2. I dataset sono pronti e puliti in: {HF_OUT_DIR}")

📦 AVVIO ESPORTAZIONE HUGGING FACE DATASETS...

🏗️  Costruzione Dataset HF: M1_detection (detection)...
   🔹 TRAIN: 2373 esempi (Skipped: 0)
   🔹 VAL: 710 esempi (Skipped: 0)
   🔹 TEST: 855 esempi (Skipped: 0)


Saving the dataset (0/1 shards):   0%|          | 0/2373 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/710 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/855 [00:00<?, ? examples/s]

   ✅ Salvato in: DATASET_ITA/PROCESSED_DATA/HF_DATASETS/M1_detection

🏗️  Costruzione Dataset HF: M2_classification (classification)...
   🔹 TRAIN: 967 esempi (Skipped: 0)
   🔹 VAL: 168 esempi (Skipped: 0)
   🔹 TEST: 211 esempi (Skipped: 0)


Saving the dataset (0/1 shards):   0%|          | 0/967 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/168 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/211 [00:00<?, ? examples/s]

   ✅ Salvato in: DATASET_ITA/PROCESSED_DATA/HF_DATASETS/M2_classification

🎉 FINE NOTEBOOK 2. I dataset sono pronti e puliti in: DATASET_ITA/PROCESSED_DATA/HF_DATASETS
